In [22]:
import cv2
import os
from tool.darknet2pytorch import Darknet
from models import Yolov4
from demo import *
from tool.utils import *
import torch.nn.utils.prune as prune
import torch
from datetime import datetime

In [3]:
cfgfile = 'cfg/yolov4-tiny.cfg'
weightfile = 'weights/yolov4-tiny.weights'
imgfile = 'data/dog.jpg'
vidfile = '../vid_short.mp4'

cap = cv2.VideoCapture(vidfile)
fps = int(cap.get(cv2.CAP_PROP_FPS))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print(height, width)
print(fps)


720 1280
24


In [8]:
# code heavily based on https://github.com/Tianxiaomo/pytorch-YOLOv4/blob/4ccef0ec8fe984e059378813e33b3740929e0c19/tool/utils.py#L99
def our_plot_boxes_cv2(img, boxes, savename=None, class_names=None, color=None, allowed_classes=None):
    img = np.copy(img)
    colors = np.array([[1, 0, 1], [0, 0, 1], [0, 1, 1], [0, 1, 0], [1, 1, 0], [1, 0, 0]], dtype=np.float32)

    def get_color(c, x, max_val):
        ratio = float(x) / max_val * 5
        i = int(math.floor(ratio))
        j = int(math.ceil(ratio))
        ratio = ratio - i
        r = (1 - ratio) * colors[i][c] + ratio * colors[j][c]
        return int(r * 255)

    width = img.shape[1]
    height = img.shape[0]
    for i in range(len(boxes)):
        box = boxes[i]
        x1 = int(box[0] * width)
        y1 = int(box[1] * height)
        x2 = int(box[2] * width)
        y2 = int(box[3] * height)

        if color:
            rgb = color
        else:
            rgb = (255, 0, 0)
        if len(box) >= 7 and class_names:
            cls_conf = box[5]
            cls_id = box[6]
            
            # skip box if it is not for a class in allowed_classes
            if allowed_classes and class_names[cls_id] not in allowed_classes:
                continue
            
#             print('%s: %f' % (class_names[cls_id], cls_conf))
            classes = len(class_names)
            offset = cls_id * 123457 % classes
            red = get_color(2, offset, classes)
            green = get_color(1, offset, classes)
            blue = get_color(0, offset, classes)
            if color is None:
                rgb = (red, green, blue)
            img = cv2.putText(img, class_names[cls_id], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1.2, rgb, 1)
        img = cv2.rectangle(img, (x1, y1), (x2, y2), rgb, 1)
    if savename:
#         print("save plot results to %s" % savename)
        cv2.imwrite(savename, img)
    return img

# code heavily based on https://github.com/Tianxiaomo/pytorch-YOLOv4/blob/4ccef0ec8fe984e059378813e33b3740929e0c19/demo.py#L25
def our_detect(cfgfile, weightfile, vidfile, use_cuda=True):
    m = Darknet(cfgfile)
    m.load_weights(weightfile)
    print('Loading weights from %s... Done!' % (weightfile))

    if use_cuda:
        m.cuda()

    num_classes = m.num_classes
    if num_classes == 20:
        namesfile = 'data/voc.names'
    elif num_classes == 80:
        namesfile = 'data/coco.names'
    else:
        namesfile = 'data/x.names'
    class_names = load_class_names(namesfile)
    cap = cv2.VideoCapture(vidfile)
    video = cv2.VideoWriter('inference_tiny.avi',cv2.VideoWriter_fourcc(*'DIVX'),24,(width,height))
    i_frame = 0
    while cap.isOpened():
        i_frame += 1
        ret, frame = cap.read()
        if not ret:
            break
        sized = cv2.resize(frame, (m.width, m.height))
        sized = cv2.cvtColor(sized, cv2.COLOR_BGR2RGB)

        for i in range(2):
            start = time.time()
            boxes = do_detect(m, sized, 0.4, 0.6, use_cuda)
            finish = time.time()
            if i == 1:
                print('Frame %s: Predicted in %f seconds.' % (i_frame, (finish - start)))

        os.makedirs('predictions', exist_ok=True)
#         plot_boxes_cv2(frame, boxes[0], savename='predictions/frame%i.jpg' % i_frame, class_names=class_names)
        frame_with_box = our_plot_boxes_cv2(frame, boxes[0], class_names=class_names, allowed_classes=['person'])
        video.write(frame_with_box)
        
#         if i_frame == 50:
#             break

    cv2.destroyAllWindows()
    video.release()

In [9]:
our_detect(cfgfile, weightfile, vidfile)

convalution havn't activate linear
convalution havn't activate linear
Loading weights from weights/yolov4-tiny.weights... Done!
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.018016
-----------------------------------
-----------------------------------
       max and argmax : 0.001002
                  nms : 0.004003
Post processing total : 0.005005
-----------------------------------
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.016014
-----------------------------------
-----------------------------------
       max and argmax : 0.001001
                  nms : 0.001001
Post processing total : 0.002002
-----------------------------------
Frame 1: Predicted in 0.022020 seconds.
-----------------------------------
           Preprocess : 0.002001
      Model Inference : 0.017017
-----------------------------------
-----------------------------------
       max and argmax : 0.001001
       

-----------------------------------
       max and argmax : 0.002003
                  nms : 0.001000
Post processing total : 0.003003
-----------------------------------
Frame 15: Predicted in 0.025023 seconds.
-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.019017
-----------------------------------
-----------------------------------
       max and argmax : 0.001001
                  nms : 0.001001
Post processing total : 0.002002
-----------------------------------
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.018016
-----------------------------------
-----------------------------------
       max and argmax : 0.001001
                  nms : 0.002002
Post processing total : 0.003002
-----------------------------------
Frame 16: Predicted in 0.024022 seconds.
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.017016
---------------------------

           Preprocess : 0.002002
      Model Inference : 0.016015
-----------------------------------
-----------------------------------
       max and argmax : 0.002002
                  nms : 0.001001
Post processing total : 0.003003
-----------------------------------
Frame 30: Predicted in 0.023021 seconds.
-----------------------------------
           Preprocess : 0.001001
      Model Inference : 0.017016
-----------------------------------
-----------------------------------
       max and argmax : 0.001001
                  nms : 0.002002
Post processing total : 0.003002
-----------------------------------
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.016014
-----------------------------------
-----------------------------------
       max and argmax : 0.001001
                  nms : 0.002003
Post processing total : 0.003004
-----------------------------------
Frame 31: Predicted in 0.022021 seconds.
---------------------------

-----------------------------------
       max and argmax : 0.002002
                  nms : 0.002002
Post processing total : 0.004004
-----------------------------------
Frame 43: Predicted in 0.023021 seconds.
-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.021019
-----------------------------------
-----------------------------------
       max and argmax : 0.001001
                  nms : 0.001001
Post processing total : 0.002002
-----------------------------------
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.016015
-----------------------------------
-----------------------------------
       max and argmax : 0.001001
                  nms : 0.001001
Post processing total : 0.002002
-----------------------------------
Frame 44: Predicted in 0.022020 seconds.
-----------------------------------
           Preprocess : 0.001001
      Model Inference : 0.016015
---------------------------

-----------------------------------
       max and argmax : 0.001002
                  nms : 0.002001
Post processing total : 0.003003
-----------------------------------
Frame 56: Predicted in 0.024022 seconds.
-----------------------------------
           Preprocess : 0.002003
      Model Inference : 0.019016
-----------------------------------
-----------------------------------
       max and argmax : 0.001002
                  nms : 0.001001
Post processing total : 0.002002
-----------------------------------
-----------------------------------
           Preprocess : 0.002003
      Model Inference : 0.019016
-----------------------------------
-----------------------------------
       max and argmax : 0.002002
                  nms : 0.001000
Post processing total : 0.003003
-----------------------------------
Frame 57: Predicted in 0.025023 seconds.
-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.018017
---------------------------

...

-----------------------------------
       max and argmax : 0.002002
                  nms : 0.002002
Post processing total : 0.004004
-----------------------------------
-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.022020
-----------------------------------
-----------------------------------
       max and argmax : 0.001001
                  nms : 0.001001
Post processing total : 0.002002
-----------------------------------
Frame 415: Predicted in 0.029026 seconds.
-----------------------------------
           Preprocess : 0.002012
      Model Inference : 0.021009
-----------------------------------
-----------------------------------
       max and argmax : 0.006006
                  nms : 0.002002
Post processing total : 0.008007
-----------------------------------
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.021019
-----------------------------------
-------------------------------

-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.029026
-----------------------------------
-----------------------------------
       max and argmax : 0.001001
                  nms : 0.002002
Post processing total : 0.003003
-----------------------------------
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.020018
-----------------------------------
-----------------------------------
       max and argmax : 0.005005
                  nms : 0.002002
Post processing total : 0.007006
-----------------------------------
Frame 430: Predicted in 0.031029 seconds.
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.022020
-----------------------------------
-----------------------------------
       max and argmax : 0.001001
                  nms : 0.001001
Post processing total : 0.002002
-----------------------------------
-------------------------------

-----------------------------------
       max and argmax : 0.005996
                  nms : 0.001001
Post processing total : 0.006997
-----------------------------------
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.022020
-----------------------------------
-----------------------------------
       max and argmax : 0.002002
                  nms : 0.001001
Post processing total : 0.003003
-----------------------------------
Frame 445: Predicted in 0.029027 seconds.
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.025023
-----------------------------------
-----------------------------------
       max and argmax : 0.001001
                  nms : 0.002002
Post processing total : 0.003003
-----------------------------------
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.025023
-----------------------------------
-------------------------------

-----------------------------------
       max and argmax : 0.001001
                  nms : 0.001000
Post processing total : 0.002002
-----------------------------------
-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.024022
-----------------------------------
-----------------------------------
       max and argmax : 0.001001
                  nms : 0.001000
Post processing total : 0.002002
-----------------------------------
Frame 460: Predicted in 0.030027 seconds.
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.021019
-----------------------------------
-----------------------------------
       max and argmax : 0.001001
                  nms : 0.001001
Post processing total : 0.002002
-----------------------------------
-----------------------------------
           Preprocess : 0.002002
      Model Inference : 0.022019
-----------------------------------
-------------------------------

-----------------------------------
       max and argmax : 0.005005
                  nms : 0.002002
Post processing total : 0.007006
-----------------------------------
Frame 473: Predicted in 0.029026 seconds.
-----------------------------------
           Preprocess : 0.002003
      Model Inference : 0.023021
-----------------------------------
-----------------------------------
       max and argmax : 0.001001
                  nms : 0.002002
Post processing total : 0.003003
-----------------------------------
-----------------------------------
           Preprocess : 0.001001
      Model Inference : 0.025022
-----------------------------------
-----------------------------------
       max and argmax : 0.001001
                  nms : 0.001001
Post processing total : 0.002002
-----------------------------------
Frame 474: Predicted in 0.031029 seconds.
-----------------------------------
           Preprocess : 0.003003
      Model Inference : 0.025023
-------------------------

In [4]:
m = Darknet(cfgfile)
m.load_weights(weightfile)


convalution havn't activate linear
convalution havn't activate linear


In [25]:
for name, param in m.named_parameters():
    raise
    prune.random_unstructured(param, name="weight", amount=0.3)
    print(name, param.size())
    raise

RuntimeError: No active exception to reraise

In [28]:
%debug

> <ipython-input-27-8018ce2bc571>(2)<module>()
      1 for name, module in m.named_modules():
----> 2     raise

ipdb> type(module)
<class 'tool.darknet2pytorch.Darknet'>
ipdb> c


In [35]:
for module in m.modules():
    print(type(module))
    raise

<class 'tool.darknet2pytorch.Darknet'>


RuntimeError: No active exception to reraise

In [52]:
for model in m.models:
    for module in model.modules():
        print(module)

Sequential(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky1): LeakyReLU(negative_slope=0.1, inplace=True)
)
Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
LeakyReLU(negative_slope=0.1, inplace=True)
Sequential(
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky2): LeakyReLU(negative_slope=0.1, inplace=True)
)
Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
LeakyReLU(negative_slope=0.1, inplace=True)
Sequential(
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

In [49]:
%debug

> <ipython-input-48-1e34e4c4c564>(2)<module>()
      1 for model in m.models:
----> 2     raise
      3     for module in model.modules():
      4         x = 1

ipdb> model.modules()
<generator object Module.modules at 0x00000197290D0F90>
ipdb> list(model.modules())
*** Error in argument: '(model.modules())'
ipdb> print(list(model.modules()))
[Sequential(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky1): LeakyReLU(negative_slope=0.1, inplace=True)
), Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False), BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), LeakyReLU(negative_slope=0.1, inplace=True)]
ipdb> c
